In [ ]:
import pyfits,glob,time,scipy
import scipy.interpolate
from scipy.special import erf
from pylab import *

topdir='/sandbox/lsst/lsst/GUI/'
thedir=topdir+'20160105'
%cd $thedir


In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

In [ ]:
ct_matrix = zeros([16,16])
# Old Matrix
ct_matrix = array([[1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0],
                    [1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0],
                    [1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0]])


fitsfilename = '114-04_spot-30um_light_352_20160105134128.fits'

xcoomin = 100
xcoomax = 400
ycoomin = 200
ycoomax = 1800

for basesegment in range(16):
    print fitsfilename
    basehdr=pyfits.getheader(fitsfilename,basesegment+1)
    baseextname = basehdr['EXTNAME']
    print basesegment, baseextname
sys.exit()


for basesegment in range(16):
    basehdr=pyfits.getheader(fitsfilename,basesegment+1)
    baseextname = basehdr['EXTNAME']
    baseimg=pyfits.getdata(fitsfilename,baseextname) 
    baseosimg=remove_overscan_xy(baseimg,509,542,2000,2022)[ycoomin:ycoomax,xcoomin:xcoomax]
    basebackground = median(baseosimg)
    for attacksegment in range(16):
        if basesegment == attacksegment or ct_matrix[basesegment,attacksegment] < 0.1:
            continue
        attackhdr=pyfits.getheader(fitsfilename,attacksegment+1)
        attackextname = attackhdr['EXTNAME']
        attackimg=pyfits.getdata(fitsfilename,attackextname) 
        attackosimg=remove_overscan_xy(attackimg,509,542,2000,2022)[ycoomin:ycoomax,xcoomin:xcoomax]
        attackbackground = median(attackosimg)
        ratio = []
    
        for i in range(0, xcoomax-xcoomin):
            for j in range(0, ycoomax-ycoomin):
                if attackextname == 'SEGMENT00':
                    if attackosimg[j,i] < 2600.0 or baseosimg[j,i] < 0.0:
                        continue
                else:
                    if attackosimg[j,i] < 12000.0 or baseosimg[j,i] < 0.0:
                        continue
                ratio.append((baseosimg[j,i]-basebackground) / (attackosimg[j,i]-attackbackground))

        if len(ratio) == 0:
            crosstalk_element = 0.0
        else:
            crosstalk_element = median(array(ratio))
        ct_matrix[basesegment,attacksegment] = crosstalk_element
        #print "Base %s, Attack %s, ratio = %.4f"%(baseextname,attackextname,crosstalk_element)
print "[",
for i in range(16):
    print "[",
    for j in range(16):
        if j < 15:
            print "%.4f,"%ct_matrix[i,j],
        else:
            print "%.4f],"%ct_matrix[i,j]
print "]"

"""
print "Nearest = %.4f"%(min(array(nearest_ratio)))
print "Next Nearest = %.4f"%(min(array(next_nearest_ratio)))
subplot(1,2,1)
hist(nearest_ratio)
subplot(1,2,2)
hist(next_nearest_ratio)
show()
"""    
        
